In [ ]:
import tensorflow as tf
import tensorflow.keras

In [ ]:
IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 3

In [ ]:
# Build the model
# input layer
inputs = tf.keras.layers.Input((IMG_WIDTH, IMG_HEIGHT, IMG_CHANNELS))

# Convert inputs from 0-255 inegers to floating decimals
s = tf.keras.layers.Lambda(lambda x: x /255)(inputs) 

# c1 layer
c1 = tf.keras.layers.Conv2D(16, (3,3), activation= "relu", kernel_initializer= "he_normal", padding= "same")(inputs)
# Add dropout before moviing on.  Drop 10%
c1 = tf.keras.Dropout(0.1)(c1)
c1 = tf.keras.layers.Conv2D(16, (3,3), activation= "relu", kernel_initializer= "he_normal", padding= "same")(c1)
# Close with a Max Pooling layer
p1 = tf.keras.layers.MaxPooling2D((2,2))(c1)

